In [1]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# checking the current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# Making connection to a Cassandra instance your local machine - (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [6]:
#Creating a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = { 'class' : 'SimpleStrategy', \
                    'replication_factor': 1 }""")
except Exception as e:
    print(e)

In [7]:
# Setting up KEYSPACE 
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [8]:
## Creating musiclibry_details table with 'sessionid' as PARTITION KEY and 'iteminsession','artist' as CLUSTERING
## COLUMNS. Here 'sessionid' is used to uniquely identify partitions. 'iteminsession' and 'artist' columns are used
## uniquely identify the rows in the partitions and sort the data accordingly

query = "CREATE TABLE IF NOT EXISTS musiclibry_details "
query = query + "(sessionid int, iteminsession int, artist text, song text, length float, \
                    PRIMARY KEY (sessionid, iteminsession, artist))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# setting up the CSV file below
file = 'event_datafile_new.csv'

# Opening the CSV file and reading it..
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        
## Creating INSERT statement below for data...
        query = "INSERT INTO musiclibry_details (sessionid, iteminsession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
    # Assigning columns from the file to the respective columns in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

## We have SELECT statement to retrieve the columns 'artist','song' and 'length' which are with sessionid = 338 
## and iteminsession = 4

query = "SELECT artist, song, length FROM musiclibry_details WHERE sessionid = 338 AND iteminsession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [9]:
## Creating songplaylists_session table with 'userid' and 'sessionid' as PARTITION KEYS and 'iteminsession' as
## CLUSTERING COLUMNS. Here partitions are uniquely identified by 'userid','sessionid', while 'iteminsession' 
## column is used as unique identify the rows in the partitions and sort the data within the partition.

query = "CREATE TABLE IF NOT EXISTS songplaylists_session"
query = query + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, \
                    lastname text, PRIMARY KEY ((userid, sessionid), iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e) 

# Setting up the CSV file 
file = 'event_datafile_new.csv'

#Opening and reading the CSV file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        
## INSERTING the values into the  table created above
        query = "INSERT INTO songplaylists_session (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
## Assigning the appropriate columns from the CSV file into the table using INSERT statement
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

## We have SELECT statement to retrieve details about 'artist', 'song', 'firstname', 'lastname' whose userid is 10
## and sessionid = 182. Here song column is sorted by iteminsession as CLUSTERING COLUMN field...

query = "SELECT artist, song, firstname, lastname FROM songplaylists_session WHERE userid = 10 AND sessionid = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [10]:
## Creating table usersongs_details with 'song' as PARTITION KEY and userid as CLUSTERING COLUMN. Here 'song' field
## will identify the partition uniquely. Where as 'userid' will uniquely identify the partition data and will sort 
## the data within the partition.

query = "CREATE TABLE IF NOT EXISTS usersongs_details "
query = query + "(song text, userid int, firstname text, lastname text, PRIMARY KEY (song, userid))"

try:
    session.execute(query)
except Exception as e:
    print(e) 

## Setting up the new CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERTING data into the above usersongs_details table
        query = "INSERT INTO usersongs_details (song, userid, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
## Assigning the appropriate columns from the CSV file into the INSERT statement for song_user_details table
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## We have SELECT statement to retrieve firstname, lastname of the users who listens to song = 'All Hands Against
## His Own'

query = "SELECT firstname, lastname FROM table_usersongs WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [11]:
## Dropping the table musiclibry_details 

query = "DROP TABLE IF EXISTS musiclibry_details"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
## Dropping table songplaylists_session

query = "DROP TABLE IF EXISTS songplaylists_session"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
## Dropping table usersongs_details

query = "DROP TABLE IF EXISTS usersongs_details"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
session.shutdown()
cluster.shutdown()